In [5]:
pip install scikit-learn==1.5.2

     --------------------------------------- 11.0/11.0 MB 81.9 MB/s eta 0:00:00
     --------------------------------------- 44.8/44.8 MB 28.4 MB/s eta 0:00:00
     ---------------------------------------- 301.8/301.8 KB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Usuario\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder

In [9]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [10]:
#One hotencoder train
# Columnas categóricas a codificar (ajusta esto según tus columnas)
columnas_categoricas = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
# Crear un objeto OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
# Aplicar el encoding a las columnas categóricas seleccionadas
encoded_data = encoder.fit_transform(train[columnas_categoricas])
# Obtener los nuevos nombres de las columnas
new_column_names = encoder.get_feature_names_out(columnas_categoricas)
# Convertir la matriz codificada en un DataFrame y concatenarlo con el DataFrame original
encoded_train = pd.DataFrame(encoded_data.toarray(), columns=new_column_names)
train = pd.concat([train.drop(columnas_categoricas, axis=1), encoded_train], axis=1)
#print(train.head())

In [11]:
#One hotencoder test
# Columnas categóricas a codificar (ajusta esto según tus columnas)
columnas_categoricas = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
# Crear un objeto OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
# Aplicar el encoding a las columnas categóricas seleccionadas
encoded_data = encoder.fit_transform(test[columnas_categoricas])
# Obtener los nuevos nombres de las columnas
new_column_names = encoder.get_feature_names_out(columnas_categoricas)
# Convertir la matriz codificada en un DataFrame y concatenarlo con el DataFrame original
encoded_test = pd.DataFrame(encoded_data.toarray(), columns=new_column_names)
test = pd.concat([test.drop(columnas_categoricas, axis=1), encoded_test], axis=1)
#print(test.head())

In [12]:
train_df = train.copy()
test_df = test.copy()

In [13]:
# Separar características y objetivo
X = train.drop(columns=['loan_status'])
y = train['loan_status']

# Normalizar datos
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# Dividir el conjunto de datos de entrenamiento para evaluación interna
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

accuracy_val = accuracy_score(y_val, y_pred_val)
precision_val = precision_score(y_val, y_pred_val)
recall_val = recall_score(y_val, y_pred_val)
f1_val = f1_score(y_val, y_pred_val)
roc_auc_val = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])

# Imprimir resultados de evaluación
print(f'Accuracy Validation: {accuracy_val:.3f}')
print(f'Precision Validation: {precision_val:.3f}')
print(f'Recall Validation: {recall_val:.3f}')
print(f'F1 Score Validation: {f1_val:.3f}')
print(f'ROC AUC Validation: {roc_auc_val:.3f}')

# Realizar predicciones en el conjunto de prueba
test_predictions = model.predict_proba(test_scaled)[:, 1]

# Crear el DataFrame de envío
submission = pd.DataFrame({'idfila': test.index, 'loan_status': test_predictions})
submission.to_csv('submission.csv', index=False)


print('Predicciones guardadas en submission.csv')

Accuracy Validation: 0.953
Precision Validation: 0.927
Recall Validation: 0.717
F1 Score Validation: 0.809
ROC AUC Validation: 0.937
Predicciones guardadas en submission.csv


In [14]:
# Realizar predicciones en el conjunto de prueba
test_predictions = model.predict_proba(test_scaled)[:, 1]

# Crear el DataFrame de envío con 'idfila' y 'loan_status'
submission = pd.DataFrame({'idfila': test.index, 'loan_status': test_predictions})

# Mostrar el DataFrame
print(submission.head())


   idfila  loan_status
0       0         0.90
1       1         0.14
2       2         0.67
3       3         0.11
4       4         0.22


In [11]:
import joblib

# Guardar el modelo en formato .pkl
joblib.dump(model, 'modelo_prestamo.pkl')
print('Modelo guardado como modelo_prestamo.pkl')


Modelo guardado como modelo_prestamo.pkl
